In [1]:
# Functions required

# XML PARSER
def get_list(path):
    tree=ET.parse(path)
    root = tree.getroot()
    text_list = []
    opinion_list = []
    for review in root.findall('Review'):
        text_string=""
        opinion_inner_list=[]
        for sent in review.findall('./sentences/sentence'):
            text_string= text_string+ " "+ sent.find('text').text
        text_list.append(text_string)
        for opinion in review.findall('./Opinions/Opinion'):
            opinion_dict = {
                opinion.get('category').replace('#','_'): opinion.get('polarity')
            }
            opinion_inner_list.append(opinion_dict)
        opinion_list.append(opinion_inner_list)
    return text_list,opinion_list



def get_most_common_aspect(opinion_list):
    import nltk
    opinion= []
    for inner_list in opinion_list:
        for _dict in inner_list:
            for key in _dict:
                opinion.append(key)
    most_common_aspect = [k for k,v in nltk.FreqDist(opinion).most_common(20)]
    return most_common_aspect


def get_data_frame(text_list,opinion_list,most_common_aspect):
    data={'Review':text_list}
    df = pd.DataFrame(data)
    if opinion_list:
        for inner_list in opinion_list:
            for _dict in inner_list:
                for key in _dict:
                    if key in most_common_aspect:
                        df.loc[opinion_list.index(inner_list),key]=_dict[key]
    return df


def get_aspect_data_frame(df,most_common_aspect):
    for common_aspect in most_common_aspect:
        df[common_aspect]=df[common_aspect].replace(['positive','negative','neutral','conflict'],[1,1,1,1])
    df = df.fillna(0)
    return df


def get_positive_data_frame(df,most_common_aspect):
    for common_aspect in most_common_aspect:
        df[common_aspect]=df[common_aspect].replace(['positive'],[1])
        df[common_aspect]=df[common_aspect].replace(['negative','neutral','conflict'],[0,0,0])
    df = df.fillna(0)
    return df


def get_negative_data_frame(df,most_common_aspect):
    for common_aspect in most_common_aspect:
        df[common_aspect]=df[common_aspect].replace(['negative'],[1])
        df[common_aspect]=df[common_aspect].replace(['positive','neutral','conflict'],[0,0,0])
    df = df.fillna(0)
    return df


def get_neutral_data_frame(df,most_common_aspect):
    for common_aspect in most_common_aspect:
        df[common_aspect]=df[common_aspect].replace(['neutral','conflict'],[1,1])
        df[common_aspect]=df[common_aspect].replace(['negative','positive'],[0,0])
    df = df.fillna(0)
    return df

def posTag(review):
    tagged_text_list=[]
    for text in review:
        tagged_text_list.append(stanford_tag.tag(word_tokenize(text)))
    return tagged_text_list


def filterTag(tagged_review):
    final_text_list=[]
    for text_list in tagged_review:
        final_text=[]
        for word,tag in text_list:
            if tag in ['NN','NNS','NNP','NNPS','RB','RBR','RBS','JJ','JJR','JJS','VB','VBD','VBG','VBN','VBP','VBZ']:
                final_text.append(word)
        final_text_list.append(' '.join(final_text))
    return final_text_list


def get_dict_aspect(y,most_common_aspect):
    position=[]
    for innerlist in y:
        position.append([i for i, j in enumerate(innerlist) if j == 1])
    sorted_common=sorted(most_common_aspect)
    dict_aspect=[]
    for innerlist in position:
        inner_dict={}
        for word in sorted_common:
            if sorted_common.index(word) in innerlist:
                inner_dict[word]= 5
            else:
                inner_dict[word]=0
        dict_aspect.append(inner_dict)
    return dict_aspect




In [2]:
import pandas as pd
from sklearn.externals import joblib
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
import xml.etree.ElementTree as ET
from lxml import etree
from scipy.sparse import hstack
import numpy as np
import warnings

In [3]:
training_path = "ABSA16_Laptops_Train_English_SB2.xml"
testing_path = "EN_LAPT_SB2_TEST.xml"

In [4]:
home = 'stanford-postagger-3.9.2-src.jar'
from nltk.tag.stanford import StanfordPOSTagger as POS_Tag
from nltk import word_tokenize
_path_to_model =  'models/english-bidirectional-distsim.tagger' 
_path_to_jar =  'stanford-postagger.jar'
stanford_tag = POS_Tag(model_filename=_path_to_model, path_to_jar=_path_to_jar)

In [5]:
train_text_list,train_opinion_list = get_list(training_path)
most_common_aspect = get_most_common_aspect(train_opinion_list)
tagged_text_list_train=joblib.load('tagged_text_list_train.pkl')
final_train_text_list=filterTag(tagged_text_list_train)

df_train = get_data_frame(final_train_text_list,train_opinion_list,most_common_aspect)
df_train_aspect = get_aspect_data_frame(df_train,most_common_aspect)
df_train_aspect = df_train_aspect.reindex_axis(sorted(df_train_aspect.columns), axis=1)


test_text_list,test_opinion_list = get_list(testing_path)

tagged_text_list_test=joblib.load('tagged_text_list_test.pkl')

final_test_text_list=filterTag(tagged_text_list_test)


df_test = get_data_frame(final_test_text_list,test_opinion_list,most_common_aspect)
df_test_aspect = get_aspect_data_frame(df_test,most_common_aspect)
df_test_aspect = df_test_aspect.reindex_axis(sorted(df_test_aspect.columns), axis=1)

X_train= df_train_aspect.Review
y_train = df_train_aspect.drop('Review',1)


X_test = df_test_aspect.Review
y_test = df_test_aspect.drop('Review',1)

import numpy as np
y_train = np.asarray(y_train, dtype=np.int64)
y_test = np.asarray(y_test, dtype=np.int64)



/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:20: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.


In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
vect = CountVectorizer(max_df=1.0,stop_words='english')  
X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)

In [7]:
nb_classif = OneVsRestClassifier(MultinomialNB()).fit(X_train_dtm, y_train)
C = 1.0 #SVregularization parameter
svc = OneVsRestClassifier(svm.SVC(kernel='linear', C=C)).fit(X_train_dtm, y_train)
lin_svc = OneVsRestClassifier(svm.LinearSVC(C=C)).fit(X_train_dtm, y_train)
sgd = OneVsRestClassifier(SGDClassifier()).fit(X_train_dtm,y_train)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:144: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


In [8]:
y_pred_class = nb_classif.predict(X_test_dtm)
y_pred_class_svc = svc.predict(X_test_dtm)
y_pred_class_lin_svc = lin_svc.predict(X_test_dtm)
y_pred_class_sgd = sgd.predict(X_test_dtm)

In [9]:
from sklearn import metrics

In [10]:
print(metrics.accuracy_score(y_test,y_pred_class))
print(metrics.accuracy_score(y_test,y_pred_class_svc))
print(metrics.accuracy_score(y_test,y_pred_class_lin_svc))
print(metrics.accuracy_score(y_test,y_pred_class_sgd))
print(metrics.precision_score(y_test,y_pred_class,average='micro'))
print(metrics.precision_score(y_test,y_pred_class_svc,average='micro'))
print(metrics.precision_score(y_test,y_pred_class_lin_svc,average='micro'))
print(metrics.precision_score(y_test,y_pred_class_sgd,average='micro'))
print(metrics.recall_score(y_test,y_pred_class,average='micro'))
print(metrics.recall_score(y_test,y_pred_class_svc,average='micro'))
print(metrics.recall_score(y_test,y_pred_class_lin_svc,average='micro'))
print(metrics.recall_score(y_test,y_pred_class_sgd,average='micro'))
print(metrics.f1_score(y_test,y_pred_class,average='micro'))
print(metrics.f1_score(y_test,y_pred_class_svc,average='micro'))
print(metrics.f1_score(y_test,y_pred_class_lin_svc,average='micro'))
print(metrics.f1_score(y_test,y_pred_class_sgd,average='micro'))
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    print(metrics.classification_report(y_test, y_pred_class))
    print(metrics.classification_report(y_test, y_pred_class_svc))
    print(metrics.classification_report(y_test, y_pred_class_lin_svc))
    print(metrics.classification_report(y_test, y_pred_class_sgd))

0.025
0.05
0.05
0.0125
0.75
0.7112299465240641
0.7321937321937322
0.6807817589576547
0.4576271186440678
0.6440677966101694
0.6222760290556901
0.5060532687651331
0.5684210526315788
0.6759847522236341
0.6727748691099477
0.5805555555555555
              precision    recall  f1-score   support

           0       0.67      0.14      0.24        14
           1       0.71      0.50      0.59        24
           2       0.00      0.00      0.00        12
           3       0.00      0.00      0.00         4
           4       0.00      0.00      0.00        21
           5       0.00      0.00      0.00         8
           6       0.00      0.00      0.00         7
           7       0.76      0.64      0.69        39
           8       1.00      1.00      1.00        80
           9       0.44      0.17      0.24        24
          10       0.62      0.70      0.65        46
          11       0.00      0.00      0.00         5
          12       0.57      0.30      0.39        27
      

In [11]:
#Stage 2:
#Generating extra feature that indicates which aspect category is present in the review
train_dict_aspect=get_dict_aspect(y_train, most_common_aspect)
d_train=DictVectorizer() 
X_train_aspect_dtm = d_train.fit_transform(train_dict_aspect)

#y_test is used to generated extra feature in order to test the performance of 2nd classifer.
#Use y_pred_class_svc(Highest performer for aspect classification) as input for extra feature to test the overall performace.
test_dict_aspect=get_dict_aspect(y_test,most_common_aspect)
d_test=DictVectorizer() 
X_test_aspect_dtm = d_test.fit_transform(test_dict_aspect)

In [12]:
def classify_sentiment(df_train,df_test,X_train_aspect_dtm,X_test_aspect_dtm):
    
    df_train = df_train.reindex_axis(sorted(df_train_positive.columns), axis=1)
    df_test = df_test.reindex_axis(sorted(df_test_positive.columns), axis=1)

    import numpy as np
    X_train = df_train.Review
    y_train = df_train.drop('Review',1)
    y_train = np.asarray(y_train, dtype=np.int64)

    X_test = df_test.Review
    y_test = df_test.drop('Review',1)
    y_test = np.asarray(y_test, dtype=np.int64)

    vect_sen = CountVectorizer(stop_words='english',ngram_range=(1,2))  
    X_train_dtm = vect_sen.fit_transform(X_train)
    X_test_dtm = vect_sen.transform(X_test)

    #ombining word vector with extra feature.
    from scipy.sparse import hstack
    X_train_dtm=hstack((X_train_dtm, X_train_aspect_dtm))
    X_test_dtm=hstack((X_test_dtm, X_test_aspect_dtm))

    C = 1.0 #SVregularization parameter
    nb_classif = OneVsRestClassifier(MultinomialNB()).fit(X_train_dtm, y_train)
    svc = OneVsRestClassifier(svm.SVC(kernel='linear', C=C)).fit(X_train_dtm, y_train)
    lin_svc = OneVsRestClassifier(svm.LinearSVC(C=C)).fit(X_train_dtm, y_train)
    sgd = OneVsRestClassifier(SGDClassifier()).fit(X_train_dtm,y_train)

    y_pred_class= nb_classif.predict(X_test_dtm)
    y_pred_class_svc = svc.predict(X_test_dtm)
    y_pred_class_lin_svc = lin_svc.predict(X_test_dtm)
    y_pred_class_sgd = sgd.predict(X_test_dtm)
    return (y_test,y_pred_class,y_pred_class_svc,y_pred_class_lin_svc,y_pred_class_sgd)

In [13]:
def print_metrices(y_test,y_pred_class,y_pred_class_svc,y_pred_class_lin_svc,y_pred_class_sgd):
    print("Accuracy:")
    print(metrics.accuracy_score(y_test,y_pred_class))
    print(metrics.accuracy_score(y_test,y_pred_class_svc))
    print(metrics.accuracy_score(y_test,y_pred_class_lin_svc))
    print(metrics.accuracy_score(y_test,y_pred_class_sgd))

    print("\nAverage precision:")
    print(metrics.precision_score(y_test,y_pred_class,average='micro'))
    print(metrics.precision_score(y_test,y_pred_class_svc,average='micro'))
    print(metrics.precision_score(y_test,y_pred_class_lin_svc,average='micro'))
    print(metrics.precision_score(y_test,y_pred_class_sgd,average='micro'))

    print("\nAverage recall:")
    print(metrics.recall_score(y_test,y_pred_class,average='micro'))
    print(metrics.recall_score(y_test,y_pred_class_svc,average='micro'))
    print(metrics.recall_score(y_test,y_pred_class_lin_svc,average='micro'))
    print(metrics.recall_score(y_test,y_pred_class_sgd,average='micro'))
    
    print("\nAverage f1:")
    print(metrics.f1_score(y_test,y_pred_class,average='micro'))
    print(metrics.f1_score(y_test,y_pred_class_svc,average='micro'))
    print(metrics.f1_score(y_test,y_pred_class_lin_svc,average='micro'))
    print(metrics.f1_score(y_test,y_pred_class_sgd,average='micro'))

    print("\nClassification report:")
    print(metrics.classification_report(y_test, y_pred_class))
    print(metrics.classification_report(y_test, y_pred_class_svc))
    print(metrics.classification_report(y_test, y_pred_class_lin_svc))
    print(metrics.classification_report(y_test, y_pred_class_sgd))

In [14]:
#For positive sentiment classifier
df_train = get_data_frame(final_train_text_list,train_opinion_list,most_common_aspect)
df_test = get_data_frame(final_test_text_list,test_opinion_list,most_common_aspect)

df_train_positive = get_positive_data_frame(df_train,most_common_aspect)
df_test_positive = get_positive_data_frame(df_test,most_common_aspect)
y_test_pos,y_pred_class_pos,y_pred_class_svc_pos,y_pred_class_lin_svc_pos,y_pred_class_sgd_pos=classify_sentiment(df_train_positive,df_test_positive,X_train_aspect_dtm,X_test_aspect_dtm)
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    print_metrices(y_test_pos,y_pred_class_pos,y_pred_class_svc_pos,y_pred_class_lin_svc_pos,y_pred_class_sgd_pos)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:144: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


Accuracy:
0.15
0.35
0.3125
0.2375

Average precision:
0.8571428571428571
0.7457627118644068
0.7560975609756098
0.7206477732793523

Average recall:
0.2608695652173913
0.7971014492753623
0.6739130434782609
0.644927536231884

Average f1:
0.4
0.7705779334500876
0.7126436781609194
0.6806883365200764

Classification report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        12
           1       0.00      0.00      0.00        14
           2       0.00      0.00      0.00         6
           3       0.00      0.00      0.00         3
           4       0.00      0.00      0.00        15
           5       0.00      0.00      0.00         7
           6       0.00      0.00      0.00         4
           7       0.00      0.00      0.00        26
           8       0.83      0.95      0.89        61
           9       0.00      0.00      0.00        11
          10       1.00      0.39      0.56        36
          11       0.00      0.

In [15]:
#For negative sentiment classifier
df_train = get_data_frame(final_train_text_list,train_opinion_list,most_common_aspect)
df_test = get_data_frame(final_test_text_list,test_opinion_list,most_common_aspect)

df_train_neg = get_negative_data_frame(df_train,most_common_aspect)
df_test_neg = get_negative_data_frame(df_test,most_common_aspect)

y_test_neg,y_pred_class_neg,y_pred_class_svc_neg,y_pred_class_lin_svc_neg,y_pred_class_sgd_neg=classify_sentiment(df_train_neg,df_test_neg,X_train_aspect_dtm,X_test_aspect_dtm)
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    print_metrices(y_test_neg,y_pred_class_neg,y_pred_class_svc_neg,y_pred_class_lin_svc_neg,y_pred_class_sgd_neg)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:144: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


Accuracy:
0.4875
0.4875
0.4625
0.45

Average precision:
0.7
0.625
0.6666666666666666
0.66

Average recall:
0.06422018348623854
0.41284403669724773
0.3302752293577982
0.30275229357798167

Average f1:
0.11764705882352941
0.49723756906077354
0.441717791411043
0.41509433962264153

Classification report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.00      0.00      0.00        10
           2       0.00      0.00      0.00         2
           3       0.00      0.00      0.00         1
           4       0.00      0.00      0.00         4
           5       0.00      0.00      0.00         1
           6       0.00      0.00      0.00         3
           7       0.00      0.00      0.00         3
           8       0.67      0.33      0.44        18
           9       0.00      0.00      0.00         9
          10       0.00      0.00      0.00        10
          11       0.00      0.00      0.00       

In [16]:
#For neutral or conflict sentiment classifier
df_train = get_data_frame(final_train_text_list,train_opinion_list,most_common_aspect)
df_test = get_data_frame(final_test_text_list,test_opinion_list,most_common_aspect)

df_train_neu = get_neutral_data_frame(df_train,most_common_aspect)
df_test_neu = get_neutral_data_frame(df_test,most_common_aspect)

y_test_neu,y_pred_class_neu,y_pred_class_svc_neu,y_pred_class_lin_svc_neu,y_pred_class_sgd_neu=classify_sentiment(df_train_neu,df_test_neu,X_train_aspect_dtm,X_test_aspect_dtm)
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    print_metrices(y_test_neu,y_pred_class_neu,y_pred_class_svc_neu,y_pred_class_lin_svc_neu,y_pred_class_sgd_neu)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:144: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


Accuracy:
0.7375
0.725
0.7375
0.75

Average precision:
0.0
0.15384615384615385
0.3333333333333333
1.0

Average recall:
0.0
0.07142857142857142
0.03571428571428571
0.03571428571428571

Average f1:
0.0
0.0975609756097561
0.06451612903225806
0.0689655172413793

Classification report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         4
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         2
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00        10
           8       0.00      0.00      0.00         1
           9       0.00      0.00      0.00         4
          10       0.00      0.00      0.00         0
          11       0.00      0.00      0.00         0
          12   

In [16]:
#Aspect Based Sentiment analyis of user's input.
user_input=input("Enter a laptop review:\n\n")
#Preprocessing and vectorizing
tagged_user_input = posTag([user_input])
filter_tagged_user_input = filterTag(tagged_user_input)

user_input_series=pd.Series(filter_tagged_user_input)
user_input_series_dtm=vect.transform(user_input_series)

predict_aspect= svc.predict(user_input_series_dtm)
extra_feature=get_dict_aspect(predict_aspect, most_common_aspect)
extra_feature_dtm=DictVectorizer().fit_transform(extra_feature)
predict_aspect

Enter a laptop review:

This is my first asus laptop. So far i am really enjoying this laptop. 512GB SSD is super fast. Battery life is also good and can last very long. I have no complain on screen quality too as display supports 4k videos. Maybe that is why it costs a lot. This is an expensive laptop and it's price is very high compared to other laptops of similar specs. So, if you have no trouble paying for this laptop, it is pretty good.


array([[1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0]])

In [17]:
test_opinion_list=[]
df_test = get_data_frame(filter_tagged_user_input,test_opinion_list,most_common_aspect)
df_train = get_data_frame(final_train_text_list,train_opinion_list,most_common_aspect)

df_train_positive = get_positive_data_frame(df_train,most_common_aspect)
y_test_pos,y_pred_class_pos,y_pred_class_svc_pos,y_pred_class_lin_svc_pos,y_pred_class_sgd_pos=classify_sentiment(df_train_positive,df_test,X_train_aspect_dtm,extra_feature_dtm)

y_pred_class_svc_pos

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:144: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


array([[1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0]])

In [18]:
test_opinion_list=[]
df_test = get_data_frame(filter_tagged_user_input,test_opinion_list,most_common_aspect)
df_train = get_data_frame(final_train_text_list,train_opinion_list,most_common_aspect)

df_train_negative = get_negative_data_frame(df_train,most_common_aspect)
y_test_neg,y_pred_class_neg,y_pred_class_svc_neg,y_pred_class_lin_svc_neg,y_pred_class_sgd_neg=classify_sentiment(df_train_negative,df_test,X_train_aspect_dtm,extra_feature_dtm)

y_pred_class_svc_neg

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:144: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [25]:
test_opinion_list=[]
df_test = get_data_frame(filter_tagged_user_input,test_opinion_list,most_common_aspect)
df_train = get_data_frame(final_train_text_list,train_opinion_list,most_common_aspect)

df_train_neutral = get_neutral_data_frame(df_train,most_common_aspect)
y_test_neu,y_pred_class_neu,y_pred_class_svc_neu,y_pred_class_lin_svc_neu,y_pred_class_sgd_neu=classify_sentiment(df_train_neutral,df_test,X_train_aspect_dtm,extra_feature_dtm)

y_pred_class_svc_neu

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:144: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]])

In [19]:
index_positive=[]
for i, (a, b) in enumerate(zip(predict_aspect.tolist()[0], y_pred_class_svc_pos.tolist()[0])):
    if a ==1 and b==1:
        index_positive.append(i)
index_positive  

[0, 3, 8, 10, 13]

In [20]:
index_negative=[]
for i, (a, b) in enumerate(zip(predict_aspect.tolist()[0], y_pred_class_svc_neg.tolist()[0])):
    if a ==1 and b==1:
        index_negative.append(i)
index_negative 

[]

In [21]:
index_neutral=[]
for i, (a, b) in enumerate(zip(predict_aspect.tolist()[0], y_pred_class_svc_neu.tolist()[0])):
    if a ==1 and b==1:
        index_neutral.append(i)
index_neutral   

[]

In [22]:
output=[]
if index_positive:
    for index in index_positive:
        output.append(sorted(most_common_aspect)[index]+": positive")
        
if index_negative:
    for index in index_negative:
        output.append(sorted(most_common_aspect)[index]+": negative")
        
if index_neutral:
    for index in index_neutral:
        output.append(sorted(most_common_aspect)[index]+": neutral or conflict")
        
        
output        

['BATTERY_OPERATION_PERFORMANCE: positive',
 'DISPLAY_GENERAL: positive',
 'LAPTOP_GENERAL: positive',
 'LAPTOP_OPERATION_PERFORMANCE: positive',
 'LAPTOP_QUALITY: positive']